In [15]:
import torch
from algo_reasoning.src.sampler import CLRSDataset

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load Pretrained Model

In [17]:
from algo_reasoning.src.models.network import EncodeProcessDecode
from algo_reasoning.src.lightning.AlgorithmicReasoningTask import AlgorithmicReasoningTask 
from algo_reasoning.src.specs import CLRS_30_ALGS
from algo_reasoning.src.losses.AlgorithmicReasoningLoss import AlgorithmicReasoningLoss

In [18]:
ckpt_path = "../checkpoints/Generalist/Generalist-epoch=49-val_loss=0.76.ckpt"

model = EncodeProcessDecode(CLRS_30_ALGS)
loss_fn = AlgorithmicReasoningLoss()

model = AlgorithmicReasoningTask.load_from_checkpoint(ckpt_path, model=model, loss_fn=loss_fn).model

/home/ronald/Documents/env/algo_reasoning/lib/python3.12/site-packages/lightning/fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
/home/ronald/Documents/env/alg

# Graph Fourier Analysis of Algorithmic Embeddings

## Toy Example: Graph Fourier Transform of BFS Embeddings

In [19]:
def fourier_transform(embeddings, nb_nodes):
    """
    Compute the Fourier Transform of the embeddings.

    Args:
        embeddings: a tensor of shape (batch_size, sequence_length, embedding_dim)

    Returns:
        a tensor of shape (batch_size, sequence_length, embedding_dim)
    """

    # Compute the eigenvectors of the Laplacian matrix
    # of the graph, only complete graphs are considered in the MPNN processor model.

    eigenvectors = []
    eigenvectors.append(torch.ones(nb_nodes, device=device)/torch.sqrt(torch.tensor(nb_nodes, device=device)))
    for i in range(1, nb_nodes):
        eig_v = torch.ones(nb_nodes, device=device)*-1
        eig_v[i] = nb_nodes-1

        eig_v = eig_v/torch.norm(eig_v)
        eigenvectors.append(eig_v)


    eigenvectors = torch.stack(eigenvectors, dim=0)

    eigenvalues = torch.ones(nb_nodes, device=device) * 16
    eigenvalues[0] = 0

    # Compute the Fourier Transform of the embeddings
    fourier_embeddings = eigenvectors@embeddings

    return fourier_embeddings, eigenvectors, eigenvalues

In [20]:
# Load dataset
algorithm_args = {
    'p': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
}

algorithms = ["bfs"]
nb_nodes = 16
ds = CLRSDataset(algorithms, nb_nodes, 1, 1000, seed=7, algorithms_args=algorithm_args)
obj = next(iter(ds)).to(device=device)

In [21]:
output = model(obj)
embeddings = output.hidden_embeddings
embeddings.shape

torch.Size([1, 5, 16, 128])

In [22]:
fourier_embeddings, eigenvectors, eigenvalues = fourier_transform(embeddings, nb_nodes)

In [23]:
torch.norm(eigenvectors[2, :])

tensor(1.0000, device='cuda:0')

In [25]:
laplacian = torch.diag(torch.ones(nb_nodes, device=device)*16) - torch.ones((16, 16), device=device)

In [30]:
result = torch.linalg.eig(laplacian)
eigenvalues = result.eigenvalues
eigenvectors = result.eigenvectors

In [50]:
eigenvectors[:, 15]

tensor([-0.1189+0.j,  0.3462+0.j,  0.3086+0.j, -0.3027+0.j, -0.4038+0.j,  0.0815+0.j,
        -0.0569+0.j,  0.1482+0.j,  0.1669+0.j, -0.0773+0.j, -0.0693+0.j,  0.2493+0.j,
        -0.3306+0.j, -0.3621+0.j,  0.3756+0.j,  0.0452+0.j], device='cuda:0')

In [10]:
eigenvectors@fourier_embeddings

tensor([[[[-2.3070e-01,  7.2455e-02, -4.4301e-01,  ...,  4.4498e-01,
           -1.2940e-01, -1.4472e-01],
          [ 7.3801e-02,  1.4629e+00,  1.5231e-01,  ..., -4.0338e-01,
            3.2576e-03,  9.4393e-01],
          [ 4.6021e-02,  4.1144e-01,  7.8297e-02,  ...,  3.8939e-01,
           -4.6680e-02, -1.2233e-01],
          ...,
          [ 6.5689e-02, -6.7532e-01,  1.3070e-01,  ..., -4.4014e-01,
           -1.1326e-02, -7.7330e-02],
          [ 6.7513e-02, -7.5080e-01,  1.3556e-01,  ..., -4.8578e-01,
           -8.0463e-03, -7.3156e-02],
          [ 7.0312e-02, -8.2965e-01,  1.4302e-01,  ..., -5.9903e-01,
           -3.0136e-03, -6.6751e-02]],

         [[-2.2756e-01, -5.7270e-01, -4.4992e-01,  ..., -3.8418e-01,
           -1.6312e-01, -3.0246e-01],
          [ 9.0792e-02,  2.5714e-01,  2.1447e-01,  ..., -9.7786e-02,
            7.7334e-02,  1.6251e-01],
          [ 4.6231e-02,  1.2516e-01,  9.5738e-02,  ...,  3.0683e-01,
           -2.7702e-03,  6.0550e-02],
          ...,
     